In [4]:
import warnings
warnings.filterwarnings('ignore')

import os
import gc
import joblib
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics, preprocessing
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model,load_model
from tensorflow.keras import callbacks
from tensorflow.keras import backend as K
from tensorflow.keras import utils

In [5]:
def auc(y_true, y_pred):
    def fallback_auc(y_true, y_pred):
        try:
            return metrics.roc_auc_score(y_true, y_pred)
        except:
            return 0.5
    return tf.py_function(fallback_auc, (y_true, y_pred), tf.double)
#     return metrics.roc_auc_score(y_true, y_pred)

In [6]:
def create_model(data, catcols):
    inputs = []
    outputs = []
    for c in catcols:
        num_unique_values = int(data[c].nunique())
        embed_dim = int(min(np.ceil((num_unique_values)/2), 50))
        inp = layers.Input(shape=(1,))
        out = layers.Embedding(num_unique_values+1, embed_dim, name=c)(inp)
        out = layers.SpatialDropout1D(0.3)(out)
        out = layers.Reshape(target_shape=(embed_dim, ))(out)
        inputs.append(inp)
        outputs.append(out)
        
    x = layers.Concatenate()(outputs)
    x = layers.BatchNormalization()(x)
    
    x = layers.Dense(300, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Dense(300, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    x = layers.BatchNormalization()(x)

    y = layers.Dense(2, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=y)
    
    return model
    
    

In [7]:
data_dir = '../Data/cat-in-the-dat-ii/'

In [8]:
pd.set_option('display.max_columns', 30)

In [13]:
train = pd.read_csv(f'{data_dir}train.csv')
test = pd.read_csv(f'{data_dir}test.csv')
sample = pd.read_csv(f'{data_dir}sample_submission.csv')

## A Bit of EDA

In [21]:
train.head()

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,0,0.0,0.0,0.0,F,N,Red,Trapezoid,Hamster,Russia,Bassoon,de4c57ee2,a64bc7ddf,598080a91,0256c7a4b,02e7c8990,3.0,Contributor,Hot,c,U,Pw,6.0,3.0,0
1,1,1.0,1.0,0.0,F,Y,Red,Star,Axolotl,NaN,Theremin,2bb3c3e5c,3a3a936e8,1dddb8473,52ead350c,f37df64af,3.0,Grandmaster,Warm,e,X,pE,7.0,7.0,0
2,2,0.0,1.0,0.0,F,N,Red,NaN,Hamster,Canada,Bassoon,b574c9841,708248125,5ddc9a726,745b909d1,NaN,3.0,NaN,Freezing,n,P,eN,5.0,9.0,0
3,3,NaN,0.0,0.0,F,N,Red,Circle,Hamster,Finland,Theremin,673bdf1f6,23edb8da3,3a33ef960,bdaa56dd1,f9d456e57,1.0,Novice,Lava Hot,a,C,NaN,3.0,3.0,0
4,4,0.0,NaN,0.0,T,N,Red,Triangle,Hamster,Costa Rica,NaN,777d1ac2c,3a7975e46,bc9cc2a94,NaN,c5361037c,3.0,Grandmaster,Cold,h,C,OZ,5.0,12.0,0


In [24]:
train['target'].unique()

array([0, 1])

In [27]:
len(train)

600000

In [47]:
train['day'].nunique()

7

In [49]:
len(sample) == len(test)

True

## Back to training loop

In [51]:
test

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
0,600000,0.0,0.0,0.0,F,Y,Blue,Polygon,Axolotl,Finland,Piano,52f6dd16c,147d704e4,8d857a0a1,ca9ad1d4b,fced9e114,3.0,Novice,Boiling Hot,f,U,oU,3.0,9.0
1,600001,0.0,0.0,0.0,F,Y,Red,Circle,Lion,Russia,Bassoon,691ebeae8,8653dcc2e,67a8d4ebb,060a21580,7ca8775da,1.0,Novice,Cold,n,N,NaN,2.0,8.0
2,600002,0.0,0.0,0.0,F,Y,Blue,Circle,Axolotl,Russia,Theremin,81f792c16,6cdda499e,69403e18c,165e81a00,5940334c9,1.0,Expert,Warm,i,N,DN,2.0,6.0
3,600003,1.0,0.0,0.0,F,N,Red,Polygon,Axolotl,Costa Rica,Bassoon,c9134205b,acbca4827,cb681246b,77d41330d,6fbdeefc8,1.0,Expert,Hot,m,B,AG,1.0,6.0
4,600004,0.0,0.0,1.0,F,Y,Red,Circle,NaN,Finland,Theremin,f0f100f57,6f800b9af,cd9feb5c6,2218d9dfe,2a27c8fde,1.0,Contributor,Lava Hot,o,J,DT,3.0,3.0
5,600005,0.0,0.0,0.0,NaN,Y,Red,Star,Axolotl,Costa Rica,Theremin,ce3da7e28,f8e870910,598080a91,745b909d1,30db15878,3.0,Grandmaster,Cold,e,Y,th,5.0,6.0
6,600006,0.0,0.0,0.0,F,Y,Red,Trapezoid,Axolotl,Russia,Bassoon,fb772f7e1,08dd65794,a7059911d,75a957c6d,ecbea3fb9,2.0,Novice,Lava Hot,n,U,RD,1.0,6.0
7,600007,0.0,0.0,1.0,F,N,Red,Trapezoid,Axolotl,India,Oboe,edd08fbe3,5473c547f,8c9b654fe,5bdcf7324,48ca7cb85,1.0,Grandmaster,Boiling Hot,f,NaN,pT,2.0,6.0
8,600008,0.0,0.0,0.0,T,Y,Red,Triangle,Dog,Russia,Bassoon,c0192e379,88794687e,3599d0255,8aeac2495,e663c4eec,2.0,Master,Warm,c,Y,nj,5.0,5.0
9,600009,0.0,0.0,0.0,F,N,Green,Polygon,Lion,Costa Rica,Bassoon,b2d1ecfad,0f59828ee,3ef3018d3,220190c9e,0ce4b3b6e,2.0,Novice,Hot,k,M,dp,7.0,2.0


In [52]:
test['target'] = -1
data = pd.concat([train, test]).reset_index(drop=True)

features = [x for x in train.columns if x not in ['id', 'target']]

for feat in features:
    label_enc = preprocessing.LabelEncoder()
    data[feat] = label_enc.fit_transform(data[feat].fillna('-1').
                                         astype(str).values)

In [56]:
len(train.columns)

25

In [58]:
len(test.columns)

25

In [57]:
len(data.columns)

25

In [59]:
len(train), len(test), len(data)

(600000, 400000, 1000000)

In [70]:
train = data[data.target != -1].reset_index(drop=True)
test = data[data.target == -1].reset_index(drop=True)

test_data = [test.loc[:, features].values[:, k] for k in 
             range(test.loc[:, features].values.shape[1])]

In [69]:
# len(test.loc[:, features].values[:, 0])

400000

In [67]:
# (test.loc[:, features].values.shape[1])

23

In [77]:
# (train.target.values)

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
oof_preds = np.zeros(len(train))
test_preds = np.zeros(len(test))

skf = StratifiedKFold(n_splits=50)

for train_index, test_index in skf.split(train, train.target.values):
    X_train, X_test = train.iloc[train_index, :], train.iloc[test_index, :]
    X_train = X_train.reset_index(drop=True)
    X_test = X_test.reset_index(drop=True)
    y_train, y_test = X_train.target.values, X_test.target.values
    
    model = create_model(data, features)
    model.compile(loss='binary_crossentropy', optimizer='adam',
                  metrics = [auc])
    
    X_train = [X_train.loc[:, features].values[:, k] for k in range(X_train.loc[:, features].values.shape[1])]
    
    X_test = [X_test.loc[:, features].values[:, k] for k in range(X_test.loc[:, features].values.shape[1])]
    
    
    es = callbacks.EarlyStopping(monitor='val_auc', min_delta=0.001,
                                 patience=5, verbose=1, mode='max',
                                baseline=None, restore_best_weights=True)
    rlr = callbacks.ReduceLROnPlateau(monitor='val_auc', factor=0.5,
                                    patience=3, min_lr=1e-06, mode='max',
                                    verbose=1)
    
    model.fit(X_train, utils.to_categorical(y_train), 
             validation_data=(X_test, utils.to_categorical(y_test)),
             verbose=1, batch_size=1024, callbacks=[es, rlr],
             epochs = 100)
    
    vaild_fold_preds = model.predict(X_test)[:, 1]
    test_fold_preds = model.predict(test_data)[:, 1]
    
    oof_preds[test_index] = valid_fold_preds.ravel()
    
    test_preds += test_fold_preds.ravel()
    
    print(metrics.roc_auc_score(y_test, valid_fold_preds))
    K.clear_session()

In [3]:
'hey'

'hey'

In [ ]:
print('Overall AUC = {}'.format(metrics.roc_auc_score(train.target.values,
                                                     oof_preds)))

In [ ]:
test_preds /= 50
test_ids = test.id.values
print("saving submisssion file")

submission = pd.DataFrame.from_dict({'id': test_ids,
                                    'target': test_preds})

submission.to_csv('submission.csv', index=False)